In [1]:
using Revise

In [2]:
using QuantumStatistics
using Cuba

┌ Info: Precompiling QuantumStatistics [08acf9d9-9812-4184-bb82-ed7be26454cb]
└ @ Base loading.jl:1260
┌ Warning: Package QuantumStatistics does not have Cuba in its dependencies:
│ - If you have QuantumStatistics checked out for development and have
│   added Cuba as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with QuantumStatistics
└ Loading Cuba into QuantumStatistics from project dependency, future warnings for QuantumStatistics are suppressed.


In [4]:
rs=5
kF=(9π/4)^(1/3)/rs
EF=kF^2
NF=kF/2/π^2
qTF=sqrt(NF*8π)
println("kF: ", kF)
println("NF: ", NF)
println("q_TF: ", qTF)

kF: 0.38383165853550255
NF: 0.01944513898110937
q_TF: 0.6990777111084903


In [327]:
kf(x)=(9π/4)^(1/3)/x
kf(10)

0.19191582926775128

In [318]:
function Pi0(q)
    x=q/2/kF
    if abs(x)<1e-8
        return -NF
    end
    
    if abs(x-1)>1.0e-8
        return -NF*(1/2-(x^2-1)/4x*log(abs((1+x)/(1-x))))
    else
        return -NF/2.0
    end 
end

function Rp(fp, q)
    #return (8π-fp*q^2)/(q^2-(8π-fp*q^2)*Pi0(q))
    return (8π-fp*q^2)/(q^2-(8π-fp*q^2)*Pi0(0.0))
end

function Rm(fm, q)
    return -fm/(1+fm*Pi0(q))
end

Rm (generic function with 1 method)

In [319]:
function AveRp(fp)
    function integrand(x)
        θ=x*π
        q=2*(kF)*sin(θ/2)
        return Rp(fp, q)*sin(θ)*π*2π/(4π)
        #return sin(θ)*π/2
    end
    
    result, error= Cuba.cuhre((x,f) -> f[1] = integrand(x[1]))
    #println(" Average Rp: ", result[1], " ± ", error[1])
    return result[1]
end

function AveRm(fm)
    function integrand(x)
        θ=x*π
        q=2*(kF)*sin(θ/2)
        return Rm(fm, q)*sin(θ)*π*2π/(4π)
        #return sin(θ)*π/2
    end
    
    result, error= Cuba.cuhre((x,f) -> f[1] = integrand(x[1]))
    #println(" Average Rm: ", result[1], " ± ", error[1])
    return result[1]
end

AveRm (generic function with 1 method)

In [320]:
AveRp(0.0)*NF

0.7825694125629311

In [321]:
k=kF/2
2π/k^2*log(abs((qTF^2+4k^2)/qTF^2))*NF

0.9314436567286085

In [322]:
println(AveRp(0.0))
println("Exact: ", 2π/kF^2*log(abs((qTF^2+4kF^2)/qTF^2)))
println(AveRm(1.0))

80.48997883976908
Exact: 80.48997883916938
-1.0077956718635224


In [323]:
function iteration(fp)
    oldfp=fp
    oldfm=0.0
    i=0
    while(i<10)
        i=i+1
        aRp=AveRp(oldfp)
        aRm=-aRp
        newfm=(oldfp-aRm)/3.0
        newfp=3*newfm+AveRm(newfm)
        println("f+:", -newfp*NF, "    f-: ",-newfm*NF)
        oldfp=newfp
        oldfm=newfm
    end
end

iteration (generic function with 1 method)

In [324]:
iteration(0.0)

f+:-0.4527743538107179    f-: -0.26085647085431035
f+:-0.6109618032617762    f-: -0.40153846943568394
f+:-0.642899065126661    f-: -0.44988375149049703
f+:-0.6475381197563076    f-: -0.4595809005232619
f+:-0.6481578675913215    f-: -0.4609875482807726
f+:-0.6482396089229524    f-: -0.46117543005287487
f+:-0.6482503715934993    f-: -0.4612002099676151
f+:-0.6482517883642405    f-: -0.4612034726637226
f+:-0.6482519748587715    f-: -0.4612039021565915
f+:-0.6482519994076077    f-: -0.46120395869224984
